In [2]:
#xlm-roberta-base
!pip install transformers datasets
!pip install nltk


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.5 MB 393.8 kB/s eta 0:00:04
   - -------------------------------------- 0.1/1.5 MB 563.7 kB/s eta 0:00:03
   ---- ----------------------------------- 0.2/1.5 MB 1.0 MB/s eta 0:00:02
   ------ --------------------------------- 0.3/1.5 MB 1.3 MB/s eta 0:00:01
   ---------- ----------------------------- 0.4/1.5 MB 1.8 MB/s eta 0:00:01
   -------------- ------------------------- 0.6/1.5 MB 2.2 MB/s eta 0:00:01
   ------------------ --------------------- 0.7/1.5 MB 2.3 MB/s eta 0:00:01
   ----------------------------- ---------- 1.1/1.5 MB 3.4 MB/s eta 0:00:01
   ------------------------------------- -- 1.4/1.5 MB 3.9 MB/s eta 0:00:01
   ------------------------------------- -- 1.4/1.5 MB 3.9 MB/s eta 0:00:01
   --------------------------


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
!pip install hebrew-preprocess

ERROR: Could not find a version that satisfies the requirement hebrew-preprocess (from versions: none)
ERROR: No matching distribution found for hebrew-preprocess

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
phrase = "ביממה האחרונה"
words = word_tokenize(phrase, language="hebrew")
stemmed_words = [stemmer.stem(word) for word in words]
print(stemmed_words)



LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/hebrew/[0m

  Searched in:
    - 'C:\\Users\\bloop123321/nltk_data'
    - 'f:\\shenkar\\pagmar\\venv\\nltk_data'
    - 'f:\\shenkar\\pagmar\\venv\\share\\nltk_data'
    - 'f:\\shenkar\\pagmar\\venv\\lib\\nltk_data'
    - 'C:\\Users\\bloop123321\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
